In [1]:
pip install rdflib

DEPRECATION: torchsde 0.2.5 has a non-standard dependency specifier numpy>=1.19.*; python_version >= "3.7". pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torchsde or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063



  Obtaining dependency information for rdflib from https://files.pythonhosted.org/packages/d4/b0/7b7d8b5b0d01f1a0b12cc2e5038a868ef3a15825731b8a0d776cf47566c0/rdflib-7.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for isodate<0.7.0,>=0.6.0 from https://files.pythonhosted.org/packages/b6/85/7882d311924cbcfc70b1890780763e36ff0b140c7e51c110fc59a532f087/isodate-0.6.1-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/531.9 kB ? eta -:--:--
   --- ----------------------------------- 41.0/531.9 kB 960.0 kB/s eta 0:00:01
   -------------------- ------------------- 276.5/531.9 kB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 531.9/531.9 kB 4.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/41.7 kB ? eta -:--:--
   ---------------------------------------- 41.7/41.7 kB 2.0 MB/s eta 0:00:00


In [9]:
pip show rdflib

Name: rdflibNote: you may need to restart the kernel to use updated packages.

Version: 7.0.0
Summary: RDFLib is a Python library for working with RDF, a simple yet powerful language for representing information.
Home-page: https://github.com/RDFLib/rdflib
Author: Daniel 'eikeon' Krech
Author-email: eikeon@eikeon.com
License: BSD-3-Clause
Location: C:\Users\barbu\anaconda3\Lib\site-packages
Requires: isodate, pyparsing
Required-by: SPARQLWrapper


In [10]:
from rdflib import Graph
import requests

In [11]:
g = Graph()
url = "https://raw.githubusercontent.com/motools/musicontology/master/rdf/musicontology.rdfs"

In [12]:
try:
    response = requests.get(url)
    g.parse(data=response.text, format="xml")
    print("Graph loaded successfully!")
except Exception as e:
    print("Error loading the RDF graph:", e)

Graph loaded successfully!


In [13]:
print("Number of triples in the graph:", len(g))

Number of triples in the graph: 2141


In [14]:
for s, p, o in g:
    print(s, p, o)

http://purl.org/ontology/mo/uuid http://www.w3.org/2000/01/rdf-schema#label universally unique identifier
http://purl.org/ontology/mo/ReleaseType http://www.w3.org/2000/01/rdf-schema#isDefinedBy http://purl.org/ontology/mo/
http://purl.org/ontology/mo/engineer http://www.w3.org/2003/06/sw-vocab-status/ns#term_status stable
http://purl.org/ontology/mo/exchange_item http://www.w3.org/2003/06/sw-vocab-status/ns#term_status stable
http://purl.org/ontology/mo/publication_of http://purl.org/ontology/mo/level 1
http://purl.org/ontology/mo/review http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
http://purl.org/ontology/mo/producer http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
http://purl.org/ontology/mo/free_download http://www.w3.org/2000/01/rdf-schema#isDefinedBy http://purl.org/ontology/mo/
http://purl.org/ontology/mo/artist http://www.w3.org/2000/01/rdf-schema#comment Relates a membership event with 

In [15]:
g.serialize(destination='musicontology.rdf', format='xml')
print("Graph saved to 'musicontology.rdf'")

Graph saved to 'musicontology.rdf'


In [16]:
from rdflib.plugins.sparql import prepareQuery
from SPARQLWrapper import SPARQLWrapper, JSON
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

In [17]:
query = prepareQuery("""
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX mo: <http://purl.org/ontology/mo/>

    SELECT ?track ?title
    WHERE {
        GRAPH ?g {
            ?track rdf:type mo:Track ;
                   rdfs:label ?title .
        }
    }
    ORDER BY ?title
""", initNs={"mo": "http://purl.org/ontology/mo/"})


In [18]:
query_str = str(query)
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery(query_str)
sparql.setReturnFormat(JSON)

C:\Users\barbu\anaconda3\Lib\site-packages\SPARQLWrapper\Wrapper.py:623: RuntimeWarning: not detected query type for query ''<rdflib.plugins.sparql.sparql.Query object at 0x0000023B8D90B050>''
  warnings.warn(
C:\Users\barbu\anaconda3\Lib\site-packages\SPARQLWrapper\Wrapper.py:633: RuntimeWarning: unknown query type 'None'
  warnings.warn("unknown query type '%s'" % r_queryType, RuntimeWarning)


In [19]:
results = sparql.query().convert()
print("Results:")
print(results)

QueryBadFormed: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b"Virtuoso 37000 Error SP030: SPARQL compiler, line 2: syntax error at '<' before 'rdflib.plugins.sparql.sparql'\n\nSPARQL query:\n#output-format:application/sparql-results+json\n<rdflib.plugins.sparql.sparql.Query object at 0x0000023B8D90B050>\n"

In [ ]:
for result in results["results"]["bindings"]:
    track = result["track"]["value"]
    title = result["title"]["value"]
    print(f"Track: {track}, Title: {title}")

In [ ]:
pip install sparqlwrapper

In [51]:
query = """
PREFIX mo: <http://purl.org/ontology/mo/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?property ?label ?comment
WHERE {
    ?property a ?type ;
              rdfs:label ?label .
    OPTIONAL { ?property rdfs:comment ?comment }
}
LIMIT 10
"""

sparql = SPARQLWrapper("http://localhost:3030/ds/query")
sparql.setQuery(query)
sparql.setReturnFormat(JSON)

results = sparql.query().convert()

print("Results:")
for result in results["results"]["bindings"]:
    property_uri = result["property"]["value"]
    label = result["label"]["value"]
    comment = result["comment"]["value"] if "comment" in result else "No comment"
    print(f"Property: {property_uri}, Label: {label}, Comment: {comment}")

Results:
Property: http://purl.org/ontology/mo/PublishedScore, Label: published score, Comment: A published score (subclass of MusicalManifestation)
Property: http://purl.org/ontology/mo/MusicalItem, Label: MusicalItem, Comment: A single exemplar of a musical expression.
    
For example, it could be a single exemplar of a CD. This is normally an single object (a CD) possessed by somebody.

From the FRBR final report: The entity defined as item is a concrete entity. It is in many instances a single physical object (e.g., a copy of a one-volume monograph, a single audio cassette, etc.). There are instances, however, where the entity defined as item comprises more than one physical object (e.g., a monograph issued as two separately bound volumes, a recording issued on three separate compact discs, etc.).

In terms of intellectual content and physical form, an item exemplifying a manifestation is normally the same as the manifestation itself. However, variations may occur from one item to

In [55]:
sparql_endpoint = "http://localhost:3030/ds/query"

sparql = SPARQLWrapper(sparql_endpoint)

query2 = """
PREFIX mo: <http://purl.org/ontology/mo/>

SELECT ?performer
WHERE {
  ?performer a mo:MusicArtist .
}
"""

sparql.setQuery(query2)
sparql.setReturnFormat(JSON)
results2 = sparql.query().convert()

print("Query 2 Results:")
for result in results2["results"]["bindings"]:
    performer = result["performer"]["value"]
    print(performer)

Query 2 Results:
http://example.org/artists/456


In [48]:
from rdflib.namespace import RDF, RDFS

# Property: http://purl.org/ontology/mo/PublishedScore
published_score_uri = mo["PublishedScore"]
g.add((published_score_uri, RDF.type, RDF.Property))
g.add((published_score_uri, RDFS.label, Literal("published score")))
g.add((published_score_uri, RDFS.comment, Literal("A published score (subclass of MusicalManifestation)")))

# Property: http://purl.org/ontology/mo/MusicalItem
musical_item_uri = mo["MusicalItem"]
g.add((musical_item_uri, RDF.type, RDF.Property))
g.add((musical_item_uri, RDFS.label, Literal("MusicalItem")))
g.add((musical_item_uri, RDFS.comment, Literal("A single exemplar of a musical expression.")))

data = g.serialize(format="turtle")

print("Serialized RDF data:")
print(data)


Serialized RDF data:
@prefix ns1: <http://purl.org/ontology/mo/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

<http://example.org/albums/123> a ns1:Album ;
    ns1:artist "Artist Name X" ;
    ns1:title "Album Title 1" .

<http://example.org/albums/456> a ns1:Album ;
    ns1:artist "Artist Name 2" ;
    ns1:title "Album Title 2" .

<http://example.org/albums/789> a ns1:Album ;
    ns1:artist "Artist Name 3" ;
    ns1:title "Album Title 3" .

<http://example.org/properties/MusicalItem> a ns1:MusicalItem,
        rdf:Property ;
    ns1:comment "A single exemplar of a musical expression." ;
    ns1:label "MusicalItem" .

<http://example.org/properties/PublishedScore> a ns1:PublishedScore,
        rdf:Property ;
    ns1:comment "A published score (subclass of MusicalManifestation)" ;
    ns1:label "published score" .

<http://example.org/properties/Score> a ns1:Score,
        rdf:Property ;
    ns1:comment "Here, we

In [53]:
mo = Namespace("http://purl.org/ontology/mo/")
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")

query = prepareQuery(
    """
    SELECT ?album ?artist ?title
    WHERE {
        ?album rdf:type ns1:Album ;
               ns1:artist ?artist ;
               ns1:title ?title .
    }
    """,
    initNs={"rdf": rdf, "ns1": mo}
)

results = g.query(query)

print("Query Results:")
for row in results:
    album = row.album
    artist = row.artist
    title = row.title
    print(f"Album: {album}, Artist: {artist}, Title: {title}")

Query Results:
Album: http://example.org/albums/123, Artist: Artist Name X, Title: Album Title 1
Album: http://example.org/albums/456, Artist: Artist Name 2, Title: Album Title 2
Album: http://example.org/albums/789, Artist: Artist Name 3, Title: Album Title 3


In [23]:
from rdflib import Graph, Namespace, Literal
from rdflib.plugins.sparql import prepareQuery

# Define your custom namespace
mo = Namespace("http://purl.org/ontology/mo/")
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")

# Create a new RDF graph
g = Graph()

# Parse the existing serialized RDF data into the graph
serialized_data = """
@prefix ns1: <http://purl.org/ontology/mo/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

<http://example.org/albums/123> a ns1:Album ;
    ns1:artist "Artist Name 1" ;
    ns1:title "Album Title 1" .

<http://example.org/albums/456> a ns1:Album ;
    ns1:artist "Artist Name 2" ;
    ns1:title "Album Title 2" .

ns1:MusicalItem a rdf:Property ;
    rdfs:label "MusicalItem" ;
    rdfs:comment "A single exemplar of a musical expression." .

ns1:PublishedScore a rdf:Property ;
    rdfs:label "published score" ;
    rdfs:comment "A published score (subclass of MusicalManifestation)" .
"""

g.parse(data=serialized_data, format='turtle')

# Add more data
additional_data = """
@prefix ns1: <http://purl.org/ontology/mo/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

<http://example.org/albums/789> a ns1:Album ;
    ns1:artist "Artist Name 3" ;
    ns1:title "Album Title 3" .

ns1:Score a rdf:Property ;
    rdfs:label "score" ;
    rdfs:comment "Here, we are dealing with the informational object (the MusicalExpression), not the actually 'published' score. This may be, for example, the product of an arrangement process." .
"""

g.parse(data=additional_data, format='turtle')

# Define the SPARQL query
query = prepareQuery(
    """
    SELECT ?album ?artist ?title ?property ?propertyLabel ?propertyComment
    WHERE {
        ?album rdf:type ns1:Album ;
               ns1:artist ?artist ;
               ns1:title ?title .

        OPTIONAL {
            ?album ?property ?value .
            ?property rdfs:label ?propertyLabel ;
                      rdfs:comment ?propertyComment .
        }
    }
    """,
    initNs={"rdf": rdf, "ns1": mo, "rdfs": rdfs}
)

# Execute the query
results = g.query(query)

# Print the results
print("Query Results:")
for row in results:
    album = row.album
    artist = row.artist
    title = row.title
    property_uri = row.property
    property_label = row.propertyLabel
    property_comment = row.propertyComment
    print(f"Album: {album}, Artist: {artist}, Title: {title}")
    print(f"  - Property: {property_uri}, Label: {property_label}, Comment: {property_comment}")

Query Results:
Album: http://example.org/albums/123, Artist: Artist Name 1, Title: Album Title 1
  - Property: None, Label: None, Comment: None
Album: http://example.org/albums/456, Artist: Artist Name 2, Title: Album Title 2
  - Property: None, Label: None, Comment: None
Album: http://example.org/albums/789, Artist: Artist Name 3, Title: Album Title 3
  - Property: None, Label: None, Comment: None


In [63]:
# Define your custom namespace
mo = Namespace("http://purl.org/ontology/mo/")
ex = Namespace("http://example.org/")

# Create a new RDF graph
g = Graph()

# Example data for each node
# Album 1
album_uri_1 = ex["albums/123"]
g.add((album_uri_1, RDF.type, mo.Album))
g.add((album_uri_1, mo.title, Literal("Album Title 1")))
g.add((album_uri_1, mo.artist, Literal("Artist Name 1")))

# Album 2
album_uri_2 = ex["albums/456"]
g.add((album_uri_2, RDF.type, mo.Album))
g.add((album_uri_2, mo.title, Literal("Album Title 2")))
g.add((album_uri_2, mo.artist, Literal("Artist Name 2")))

# Additional Example Data
# Album 3
album_uri_3 = ex["albums/789"]
g.add((album_uri_3, RDF.type, mo.Album))
g.add((album_uri_3, mo.title, Literal("Album Title 3")))
g.add((album_uri_3, mo.artist, Literal("Artist Name 3")))

# Property: MusicalItem
musical_item_uri = ex["properties/MusicalItem"]
g.add((musical_item_uri, RDF.type, RDF.Property))
g.add((musical_item_uri, RDF.type, mo.MusicalItem))
g.add((musical_item_uri, mo.label, Literal("MusicalItem")))
g.add((musical_item_uri, mo.comment, Literal("A single exemplar of a musical expression.")))

# Property: PublishedScore
published_score_uri = ex["properties/PublishedScore"]
g.add((published_score_uri, RDF.type, RDF.Property))
g.add((published_score_uri, RDF.type, mo.PublishedScore))
g.add((published_score_uri, mo.label, Literal("published score")))
g.add((published_score_uri, mo.comment, Literal("A published score (subclass of MusicalManifestation)")))

# Property: Score
score_uri = ex["properties/Score"]
g.add((score_uri, RDF.type, RDF.Property))
g.add((score_uri, RDF.type, mo.Score))
g.add((score_uri, mo.label, Literal("score")))
g.add((score_uri, mo.comment, Literal("Here, we are dealing with the informational object (the MusicalExpression), not the actually 'published' score. This may be, for example, the product of an arrangement process.")))

# Serialize the graph to a string in Turtle format
data = g.serialize(format="turtle")

# Print the serialized data
print("Serialized RDF data:")
print(data)  # Decode to print as string

Serialized RDF data:
@prefix ns1: <http://purl.org/ontology/mo/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .

<http://example.org/albums/123> a ns1:Album ;
    ns1:artist "Artist Name 1" ;
    ns1:title "Album Title 1" .

<http://example.org/albums/456> a ns1:Album ;
    ns1:artist "Artist Name 2" ;
    ns1:title "Album Title 2" .

<http://example.org/albums/789> a ns1:Album ;
    ns1:artist "Artist Name 3" ;
    ns1:title "Album Title 3" .

<http://example.org/properties/MusicalItem> a ns1:MusicalItem,
        rdf:Property ;
    ns1:comment "A single exemplar of a musical expression." ;
    ns1:label "MusicalItem" .

<http://example.org/properties/PublishedScore> a ns1:PublishedScore,
        rdf:Property ;
    ns1:comment "A published score (subclass of MusicalManifestation)" ;
    ns1:label "published score" .

<http://example.org/properties/Score> a ns1:Score,
        rdf:Property ;
    ns1:comment "Here, we are dealing with the informational object (the MusicalE

In [64]:
query_combined = """
SELECT ?entity ?label ?comment
WHERE {
  {
    ?entity a mo:Album ;
            mo:title ?label ;
            mo:artist ?comment .
  }
  UNION
  {
    ?entity a rdf:Property ;
            rdfs:label ?label ;
            rdfs:comment ?comment .
  }
}
"""
results_combined = g.query(query_combined, initNs={"mo": mo, "ex": ex, "rdf": RDF, "rdfs": RDFS})
for row in results_combined:
    entity, label, comment = row
    print(f"Entity: {entity}, Label: {label}, Comment: {comment}")

Entity: http://example.org/albums/123, Label: Album Title 1, Comment: Artist Name 1
Entity: http://example.org/albums/456, Label: Album Title 2, Comment: Artist Name 2
Entity: http://example.org/albums/789, Label: Album Title 3, Comment: Artist Name 3


In [45]:
# Define your custom namespace
mo = Namespace("http://purl.org/ontology/mo/")
ex = Namespace("http://example.org/")

# Create a new RDF graph
g = Graph()

# Add data for Album 1
album_uri_1 = ex["albums/123"]
g.add((album_uri_1, RDF.type, mo.Album))
g.add((album_uri_1, mo.title, Literal("Album Title 1")))
g.add((album_uri_1, mo.artist, Literal("Artist Name 1")))
g.add((album_uri_1, mo.published_score, Literal("Score 1")))
g.add((album_uri_1, mo.record_number, Literal("1")))
g.add((album_uri_1, mo.compiled, Literal("Compiler 1")))

# Add data for Album 2
album_uri_2 = ex["albums/456"]
g.add((album_uri_2, RDF.type, mo.Album))
g.add((album_uri_2, mo.title, Literal("Album Title 2")))
g.add((album_uri_2, mo.artist, Literal("Artist Name 2")))
g.add((album_uri_2, mo.published_score, Literal("Score 2")))
g.add((album_uri_2, mo.record_number, Literal("2")))
g.add((album_uri_2, mo.compiled, Literal("Compiler 2")))

# Add data for Album 3
album_uri_3 = ex["albums/789"]
g.add((album_uri_3, RDF.type, mo.Album))
g.add((album_uri_3, mo.title, Literal("Album Title 3")))
g.add((album_uri_3, mo.artist, Literal("Artist Name 3")))
g.add((album_uri_3, mo.published_score, Literal("Score 3")))
g.add((album_uri_3, mo.record_number, Literal("3")))
g.add((album_uri_3, mo.compiled, Literal("Compiler 3")))

# Serialize the graph to a string in Turtle format
data = g.serialize(format="turtle")

# Print the serialized data
print("Serialized RDF data:")
print(data) 

Serialized RDF data:
@prefix ns1: <http://purl.org/ontology/mo/> .

<http://example.org/albums/123> a ns1:Album ;
    ns1:artist "Artist Name 1" ;
    ns1:compiled "Compiler 1" ;
    ns1:published_score "Score 1" ;
    ns1:record_number "1" ;
    ns1:title "Album Title 1" .

<http://example.org/albums/456> a ns1:Album ;
    ns1:artist "Artist Name 2" ;
    ns1:compiled "Compiler 2" ;
    ns1:published_score "Score 2" ;
    ns1:record_number "2" ;
    ns1:title "Album Title 2" .

<http://example.org/albums/789> a ns1:Album ;
    ns1:artist "Artist Name 3" ;
    ns1:compiled "Compiler 3" ;
    ns1:published_score "Score 3" ;
    ns1:record_number "3" ;
    ns1:title "Album Title 3" .




In [42]:
mo = Namespace("http://purl.org/ontology/mo/")
ex = Namespace("http://example.org/")

# Define the SPARQL query for albums with their properties
query_albums_properties = prepareQuery("""
    SELECT ?album ?title ?artist ?score ?record ?compiler
    WHERE {
        ?album a mo:Album ;
               mo:title ?title ;
               mo:artist ?artist ;
               mo:published_score ?score ;
               mo:record_number ?record ;
               mo:compiled ?compiler .
    }
""")

# Execute the query and print results
results_albums_properties = g.query(query_albums_properties, initNs={"mo": mo, "ex": ex, "rdf": RDF, "rdfs": RDFS})
print("\nQuery Results:")
for row in results_albums_properties:
    album_uri, title, artist, score, record, compiler = row
    print(f"Album: {album_uri}, Artist: {artist}, Title: {title}")
    print(f"  - Score: {score}, Record Number: {record}, Compiler: {compiler}")

Exception: Unknown namespace prefix : mo

In [57]:
from rdflib.plugins.sparql import prepareQuery
from rdflib.plugins.sparql.processor import SPARQLResult

# Define the SPARQL endpoint
sparql_endpoint = "http://localhost:3030/ds/query"

# Define the SPARQL query for albums with their properties
query_albums_properties = prepareQuery("""
    SELECT ?album ?title ?artist ?score ?record ?compiler
    WHERE {
        ?album a mo:Album ;
               mo:title ?title ;
               mo:artist ?artist ;
               mo:published_score ?score ;
               mo:record_number ?record ;
               mo:compiled ?compiler .
    }
""", initNs={"mo": mo})

# Execute the query using the SPARQL endpoint
results_albums_properties = SPARQLResult.query(sparql_endpoint, query_albums_properties)

# Print the results
print("\nQuery Results:")
for row in results_albums_properties:
    album_uri, title, artist, score, record, compiler = row
    print(f"Album: {album_uri}, Title: {title}, Artist: {artist}, Score: {score}, Record: {record}, Compiler: {compiler}")

AttributeError: type object 'SPARQLResult' has no attribute 'query'

In [62]:
from rdflib import Namespace, Literal
from rdflib.namespace import RDF
from SPARQLWrapper import SPARQLWrapper, JSON

# Define your custom namespace
mo = Namespace("http://purl.org/ontology/mo/")

# Create a new RDF graph
g = Graph()

# Add data for Album 1
album_uri_1 = mo["albums/123"]
g.add((album_uri_1, RDF.type, mo.Album))
g.add((album_uri_1, mo.title, Literal("Album Title 1")))
g.add((album_uri_1, mo.artist, Literal("Artist Name 1")))
g.add((album_uri_1, mo.published_score, Literal("Score 1")))
g.add((album_uri_1, mo.record_number, Literal("1")))
g.add((album_uri_1, mo.compiled, Literal("Compiler 1")))

# Add data for Album 2
album_uri_2 = mo["albums/456"]
g.add((album_uri_2, RDF.type, mo.Album))
g.add((album_uri_2, mo.title, Literal("Album Title 2")))
g.add((album_uri_2, mo.artist, Literal("Artist Name 2")))
g.add((album_uri_2, mo.published_score, Literal("Score 2")))
g.add((album_uri_2, mo.record_number, Literal("2")))
g.add((album_uri_2, mo.compiled, Literal("Compiler 2")))

# Serialize the graph to a string in Turtle format
data = g.serialize(format="turtle")

# Print the serialized data
print("Serialized RDF data:")
print(data)

# Define the SPARQL endpoint
sparql_endpoint = "http://localhost:3030/ds/query"

# Define the SPARQL query for albums with their properties
query_albums_properties = """
    PREFIX mo: <http://purl.org/ontology/mo/>
    SELECT ?album ?title ?artist ?score ?record ?compiler
    WHERE {
        ?album a mo:Album ;
               mo:title ?title ;
               mo:artist ?artist ;
               mo:published_score ?score ;
               mo:record_number ?record ;
               mo:compiled ?compiler .
    }
"""

# Create a new SPARQLWrapper instance
sparql = SPARQLWrapper(sparql_endpoint)

# Set the query
sparql.setQuery(query_albums_properties)

# Set the return format as JSON
sparql.setReturnFormat(JSON)

# Execute the query and print the results
print("\nQuery Results:")
results_albums_properties = sparql.query().convert()
for result in results_albums_properties["results"]["bindings"]:
    album = result["album"]["value"]
    title = result["title"]["value"]
    artist = result["artist"]["value"]
    score = result["score"]["value"]
    record = result["record"]["value"]
    compiler = result["compiler"]["value"]
    print(f"Album: {album}, Title: {title}, Artist: {artist}, Score: {score}, Record: {record}, Compiler: {compiler}")


Serialized RDF data:
@prefix ns1: <http://purl.org/ontology/mo/> .

<http://purl.org/ontology/mo/albums/123> a ns1:Album ;
    ns1:artist "Artist Name 1" ;
    ns1:compiled "Compiler 1" ;
    ns1:published_score "Score 1" ;
    ns1:record_number "1" ;
    ns1:title "Album Title 1" .

<http://purl.org/ontology/mo/albums/456> a ns1:Album ;
    ns1:artist "Artist Name 2" ;
    ns1:compiled "Compiler 2" ;
    ns1:published_score "Score 2" ;
    ns1:record_number "2" ;
    ns1:title "Album Title 2" .



Query Results:
